In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from scipy.stats import spearmanr, kendalltau

In [2]:
df = pd.read_csv("marketing_advanced.csv", encoding="euc-kr")

In [3]:
df.head(10)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_timestamp,order_estimated_delivery_date,age,gender,country,...,category2,price,delivery_delay_days,approval_delay_minutes,weight_kg,volume_cm3,review_score,marketing_info_agree,agegroup,product_id
0,e22acc9c116caa3f2b7121bbb380d08e,0000366f3b9a7992bf8c76cfdf3221e2,delivered,2024-10-10 16:29,2024-10-10 19:21,2024-10-16 20:48,2024-10-21,56,남성,핀란드,...,건강식품,62.17,-5.0,15.0,5,11845.97,5,y,50대,d2e1b02d25294e6f
1,3594e05a005ac4d06a72673270ef9ec9,0000b849f77a49e4a4ce2b2a4ca5be3f,delivered,2024-10-07 17:47,2024-10-07 19:14,2024-10-10 18:02,2024-10-15,61,여성,한국,...,건강식품,57.90,-5.0,434.0,1,25981.89,4,y,50대,d2e1b02d25294e6f
2,b33ec3b699337181488304f362a6b734,0000f46a3911fa3c0805444483337064,delivered,2023-08-10 16:35,2023-08-10 19:29,2023-09-05 14:38,2023-09-07,51,남성,필리핀,...,건강식품,60.34,-2.0,0.0,0,26478.17,2,n,50대,d2e1b02d25294e6f
3,41272756ecddd9a9ed0180413cc22fb6,0000f6ccb0745a6a4b88665a16c9f078,delivered,2024-03-12 15:03,2024-03-12 17:00,2024-04-01 21:23,2024-04-13,34,여성,덴마크,...,음향가전,128.17,-12.0,20.0,10,208064.64,4,y,30대,f9614bb192ee4050
4,d957021f1127559cd947b62533f484f7,0004aac84e0df4da2b147fca70cf8255,delivered,2024-04-14 13:49,2024-04-14 15:41,2024-04-27 23:08,2024-05-05,42,남성,벨기에,...,주방가전,110.15,-8.0,21.0,5,185186.95,2,y,40대,76a306b4cd254f91
5,3e470077b690ea3e3d501cffb5e0c499,0004bd2a26a76fe21f786e4fbd80607f,delivered,2024-09-05 19:12,2024-09-05 21:23,2024-09-07 16:12,2024-09-19,60,남성,중국,...,낚세,47.11,-12.0,15.0,5,104803.74,3,y,50대,08473b6a9bd24eec
6,d0028facea13f508e880202d7097a5a1,00050ab1314c0e55a6ca13cf7181fecf,delivered,2024-09-20 17:23,2024-09-20 18:21,2024-09-27 12:08,2024-10-09,58,여성,일본,...,캠핑,40.91,-12.0,6654.0,5,79092.24,4,y,50대,4e3c726f56f94f5b
7,44e608f2db00c74a1fe329de44416a4e,00053a61a98854899e70ed204dd4bafe,delivered,2024-07-28 12:48,2024-07-28 15:15,2024-08-16 19:02,2024-08-26,54,여성,핀란드,...,낚세,48.01,-10.0,17.0,5,81180.21,2,y,50대,08473b6a9bd24eec
8,44e608f2db00c74a1fe329de44416a4e,00053a61a98854899e70ed204dd4bafe,delivered,2024-07-28 16:30,2024-07-28 16:45,2024-08-16 19:02,2024-08-26,54,여성,핀란드,...,신선식품,82.62,-10.0,17.0,10,16475.92,2,y,50대,71926bbfce2a47bc
9,ae76bef74b97bcb0b3e355e60d9a6f9c,0005e1862207bf6ccc02e4228effd9a0,delivered,2023-08-04 16:18,2023-08-04 18:52,2023-08-09 8:33,2023-09-06,38,남성,덴마크,...,파티의류,32.43,-28.0,11.0,1,59244.70,3,y,30대,64629adebdc649d6


In [4]:
df['category1'].unique()

array(['식품', '전자제품', '스포츠용품', '의류', '가구', '화장품'], dtype=object)

### 카테고리1별로 고객 1인당 평균 주문 건수

In [34]:
order_per_user = df.groupby(['category1', 'customer_id'])['order_id'].count().reset_index()
result = order_per_user.groupby('category1')['order_id'].mean().round(2)
result

category1
가구       1.04
스포츠용품    1.05
식품       1.05
의류       1.05
전자제품     1.04
화장품      1.05
Name: order_id, dtype: float64

### 카테고리1 별 평균 재주문까지 걸린 일수

In [ ]:
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp']) # 날짜 타입변환
df_sorted = df.sort_values(['customer_id', 'category1', 'order_purchase_timestamp']) # 고객아이디, 카테고리1, 구매요청시간 오름차순
df_sorted['reorder_gap'] = df_sorted.groupby(['customer_id', 'category1'])['order_purchase_timestamp'].diff().dt.days # 리오더 컬럼 생성후 고객아이디와 카테고리1을 그룹바이후 구매요청시간을 뺌
mean_gap = df_sorted.groupby('category1')['reorder_gap'].mean().round(2) #카테고리1, 리오더 그룹바이 후 평균 후 소수점 2까지
mean_gap = mean_gap.sort_values(ascending=False) # 내림차순
print("카테고리1별 재주문까지 걸린 평균 일수:\n", mean_gap)

카테고리1별 재주문까지 걸린 평균 일수:
 category1
스포츠용품    19.79
화장품      18.61
식품       18.06
전자제품     17.27
의류       16.61
가구       12.92
Name: reorder_gap, dtype: float64


In [8]:
df['category2'].unique()

array(['건강식품', '음향가전', '주방가전', '낚세', '캠핑', '신선식품', '파티의류', '가공식품',
       '책상/책장', '침대', '바디케어', '거실장/테이블', '건강가전', '유아의류', '비데', '스킨케어',
       '스포츠의류', '여성의류', '등산', '네일케어', '골프', '남성의류', '메이크업', '마스크/팩',
       '선케어', '세탁기/건조기', '청소기', '옷장/행거', '남성화장품', '헤어케어', '수납장/선반', '화장대',
       '냉장고', '1인가구', '향수', '쇼파', '야외가구'], dtype=object)

In [ ]:
order_per_user2 = df.groupby(['category2', 'customer_id'])['order_id'].count().reset_index()
result2 = order_per_user2.groupby('category2')['order_id'].mean().round(2) # 카테고리2를 오더아이디로 그룹바이후 평균 및 소수점 2까지
result2 = result2.sort_values(ascending=False) # 결과를 내림차순
result2

category2
메이크업       1.03
스킨케어       1.03
건강식품       1.02
신선식품       1.02
가공식품       1.02
파티의류       1.01
여성의류       1.01
주방가전       1.01
스포츠의류      1.01
비데         1.01
캠핑         1.01
유아의류       1.01
음향가전       1.01
등산         1.01
향수         1.01
냉장고        1.01
남성화장품      1.01
남성의류       1.01
낚세         1.01
골프         1.01
건강가전       1.01
헤어케어       1.01
침대         1.00
청소기        1.00
책상/책장      1.00
1인가구       1.00
쇼파         1.00
옷장/행거      1.00
야외가구       1.00
수납장/선반     1.00
세탁기/건조기    1.00
선케어        1.00
바디케어       1.00
마스크/팩      1.00
네일케어       1.00
거실장/테이블    1.00
화장대        1.00
Name: order_id, dtype: float64

### 카테고리2 별로 재주문까지 걸린 일수

In [ ]:
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp']) # datetime으로 변환
df_sorted = df.sort_values(['customer_id', 'category2', 'order_purchase_timestamp']) # 오름차순으로 
df_sorted['reorder_gap'] = df_sorted.groupby(['customer_id', 'category2'])['order_purchase_timestamp'].diff().dt.days # 날짜 차이 구함
mean_gap_2 = df_sorted.groupby('category2')['reorder_gap'].mean().round(2) # 카테고리2를 리오더갭으로 그룹바이 후 평균.소수점 2까지
mean_gap_2  = mean_gap_2.sort_values(ascending=False) # 결과를 내림차순
mean_gap_2

category2
헤어케어       75.50
세탁기/건조기    42.69
등산         36.10
건강가전       35.87
유아의류       32.34
향수         30.29
선케어        28.20
화장대        24.00
신선식품       21.16
여성의류       17.85
메이크업       17.11
캠핑         16.95
가공식품       14.03
침대         14.00
낚세         13.63
청소기        13.38
건강식품       13.16
파티의류       11.06
골프          9.96
스킨케어        9.02
주방가전        8.93
남성의류        8.53
스포츠의류       8.00
음향가전        7.71
마스크/팩       5.50
남성화장품       5.43
비데          2.38
냉장고         1.47
1인가구        0.00
책상/책장       0.00
옷장/행거       0.00
야외가구        0.00
바디케어        0.00
네일케어        0.00
쇼파          0.00
거실장/테이블      NaN
수납장/선반       NaN
Name: reorder_gap, dtype: float64

### 해석:
nan = 재구매를 안함
0 = 재구매는 했지만 왜인지는 모르겠으나 첫구와 재구매가 같은날 발생, 따라서 구매일 차이가 없음으로 0으로 표시됨

In [ ]:
# category2의 각 값이 category1의 어떤 값에 속하는지 유니크하게 추출
mapping = df[['category2', 'category1']].drop_duplicates() # drop_duplicates()를 사용해서 중복 없이 매핑
print(mapping)

    category2 category1
0        건강식품        식품
3        음향가전      전자제품
4        주방가전      전자제품
5          낚세     스포츠용품
6          캠핑     스포츠용품
8        신선식품        식품
9        파티의류        의류
11       가공식품        식품
12      책상/책장        가구
13         침대        가구
14       바디케어       화장품
19    거실장/테이블        가구
20       건강가전      전자제품
24       유아의류        의류
25         비데      전자제품
27       스킨케어       화장품
28      스포츠의류        의류
30       여성의류        의류
32         등산     스포츠용품
37       네일케어       화장품
38         골프     스포츠용품
41       남성의류        의류
44       메이크업       화장품
45      마스크/팩       화장품
54        선케어       화장품
62    세탁기/건조기      전자제품
63        청소기      전자제품
78      옷장/행거        가구
89      남성화장품       화장품
102      헤어케어       화장품
109    수납장/선반        가구
127       화장대        가구
166       냉장고      전자제품
189      1인가구        가구
219        향수       화장품
274        쇼파        가구
295      야외가구        가구


### 가장 오래전에 주문한 사람

In [11]:
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])

In [12]:
oldest_order = df.loc[df['order_purchase_timestamp'].idxmin()]

In [13]:
df_sorted = df.sort_values(by='order_purchase_timestamp', ascending=True)

In [14]:
df_sorted[['customer_id', 'order_purchase_timestamp']]

,customer_id,order_purchase_timestamp
75400,b7d76e111c89f7ebf14761390f0f7d17,2023-02-04 09:05:00
75399,b7d76e111c89f7ebf14761390f0f7d17,2023-02-04 18:35:00
29616,4854e9b3feff728c13ee5fc7d1547e92,2023-02-05 19:39:00
5996,0eb1ee9dba87f5b36b4613a65074337c,2023-03-02 15:58:00
6952,10e89fd8e5c745f81bec101207ba4d7d,2023-03-03 06:15:00
...,...,...
13892,21dbe8eabd00b34492a939c540e2b1a7,2025-01-29 15:35:00
15024,24ac2b4327e25baf39f2119e4228976a,2025-01-29 18:22:00
74222,b4dcade04bc548b7e3b0243c801f8c26,2025-01-29 19:31:00
1693,0421e7a23f21e5d54efed456aedbc513,2025-01-29 21:38:00


### 주문량이 많은 순서

In [15]:
order_count = df['customer_id'].value_counts().reset_index()
order_count.columns = ['customer_id', 'order_count']

In [16]:
order_count

,customer_id,order_count
0,8d50f5eadf50201ccdcedfb9e2ac8455,16
1,3e43e6105506432c953e165fb2acf44c,13
2,41a3b256cc497dc952a815b848345cbc,12
3,1b6c7548a2a1f9037c1fd3ddfed95f33,10
4,ca77025e7201e3b30c44b472ff346268,9
...,...,...
95414,58830e73978eab27a86d804befbfa3a7,1
95415,5882e10ed75dbb86ed6b62bddb146e5e,1
95416,5880e398144a5fc8dbcd185521e3f6bc,1
95417,58808b1eb6a9ebd14974101c600e6f8f,1


### 마지막 구매가 6개월(day 180) 전인 사람

In [17]:
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
latest_date = df['order_purchase_timestamp'].max()
last_purchase = df.groupby('customer_id')['order_purchase_timestamp'].max().reset_index()
six_months_ago = latest_date - pd.Timedelta(days=180)
churn_customers = last_purchase[last_purchase['order_purchase_timestamp'] <= six_months_ago]
len(churn_customers)
churn_customers

,customer_id,order_purchase_timestamp
2,0000f46a3911fa3c0805444483337064,2023-08-10 16:35:00
3,0000f6ccb0745a6a4b88665a16c9f078,2024-03-12 15:03:00
4,0004aac84e0df4da2b147fca70cf8255,2024-04-14 13:49:00
7,00053a61a98854899e70ed204dd4bafe,2024-07-28 16:30:00
8,0005e1862207bf6ccc02e4228effd9a0,2023-08-04 16:18:00
...,...,...
95412,fffbf87b7a1a6fa8b03f081c5f51a201,2024-05-27 20:04:00
95414,fffcf5a5ff07b0908bd4e2dbc735a684,2023-11-08 15:00:00
95415,fffea47cd6d3cc0a88bd621562a9d061,2024-05-10 12:45:00
95416,ffff371b4d645b6ecea244b27531430a,2023-07-07 04:15:00


### 시간대별로 카테고리1

In [18]:
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
df['hour'] = df['order_purchase_timestamp'].dt.hour

In [19]:
def time_period(hour):
    if 6 <= hour < 12:
        return '6~12'
    elif 12 <= hour < 18:
        return '12~18'
    elif 18 <= hour < 24:
        return '18~24'
    else:
        return '0~6'

In [20]:
df['time_period'] = df['hour'].apply(time_period)

In [21]:
result = df.groupby(['time_period', 'category1']).size().reset_index(name='order_count')

In [22]:
result = result.sort_values(['time_period', 'order_count'], ascending=[True, False])

In [23]:
result

,time_period,category1,order_count
2,0~6,식품,2272
3,0~6,의류,1763
1,0~6,스포츠용품,1633
4,0~6,전자제품,1628
5,0~6,화장품,1126
0,0~6,가구,560
8,12~18,식품,9321
9,12~18,의류,7454
10,12~18,전자제품,6704
7,12~18,스포츠용품,6664


In [24]:
result.sort_values

<bound method DataFrame.sort_values of    time_period category1  order_count
2          0~6        식품         2272
3          0~6        의류         1763
1          0~6     스포츠용품         1633
4          0~6      전자제품         1628
5          0~6       화장품         1126
0          0~6        가구          560
8        12~18        식품         9321
9        12~18        의류         7454
10       12~18      전자제품         6704
7        12~18     스포츠용품         6664
11       12~18       화장품         4482
6        12~18        가구         2092
14       18~24        식품        11113
15       18~24        의류         8597
13       18~24     스포츠용품         8026
16       18~24      전자제품         7853
17       18~24       화장품         5346
12       18~24        가구         2531
20        6~12        식품         4029
21        6~12        의류         3209
19        6~12     스포츠용품         2889
22        6~12      전자제품         2810
23        6~12       화장품         1855
18        6~12        가구          942>

### 카테고리1 별 구매량

In [25]:
category1_count = df.groupby('category1')['order_id'].count().reset_index()
category1_count = category1_count.rename(columns={'order_id': 'order_count'})
category1_count = category1_count.sort_values('order_count', ascending=False)
category1_count

,category1,order_count
2,식품,26735
3,의류,21023
1,스포츠용품,19212
4,전자제품,18995
5,화장품,12809
0,가구,6125


### 카테고리2 별 구매량

In [26]:
category2_count = df.groupby('category2')['order_id'].count().reset_index()
category2_count = category2_count.rename(columns={'order_id': 'order_count'})
category2_count = category2_count.sort_values('order_count', ascending=False)
category2_count

,category2,order_count
4,건강식품,9040
22,신선식품,8864
1,가공식품,8831
6,낚세,4852
32,캠핑,4826
11,등산,4824
5,골프,4710
24,여성의류,4294
26,유아의류,4263
21,스포츠의류,4168


### 이탈감지

### 충성고객

### 카테고리별 맞춤고객